In [ ]:
import pandas as pd
import os, time
import numpy as np
from tensorflow.keras.applications import MobileNetV3Small
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#MobileNetv3Small
data_dir = "data/north_american_predators/greyscale_downsampled_split/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/greyscale_downsampled_split/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/greyscale_downsampled_split/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)
print(dataset_train.class_names)

Found 13823 files belonging to 8 classes.
Found 3454 files belonging to 8 classes.
Found 4317 files belonging to 8 classes.
['black_bear', 'brown_bear', 'cougar', 'coyote', 'human', 'non_human', 'polar_bear', 'wolf']


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [4]:
base_model = tf.keras.applications.MobileNetV3Small(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
early_stop = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

start_time = time.time()
model.fit(dataset_train, epochs=100, validation_data=dataset_val, callbacks=[early_stop])
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Epoch 1/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 21s 88ms/step - loss: 0.4901 - sparse_categorical_accuracy: 0.8533 - val_loss: 0.5640 - val_sparse_categorical_accuracy: 0.8092
Epoch 2/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - loss: 0.1668 - sparse_categorical_accuracy: 0.9523 - val_loss: 0.5396 - val_sparse_categorical_accuracy: 0.8217
Epoch 3/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 17s 80ms/step - loss: 0.1203 - sparse_categorical_accuracy: 0.9662 - val_loss: 0.5400 - val_sparse_categorical_accuracy: 0.8309
Epoch 4/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 16s 76ms/step - loss: 0.0978 - sparse_categorical_accuracy: 0.9719 - val_loss: 0.5196 - val_sparse_categorical_accuracy: 0.8396
Epoch 5/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - loss: 0.0840 - sparse_categorical_accuracy: 0.9761 - val_loss: 0.5755 - val_sparse_categorical_accuracy: 0.8373
Epoch 6/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - loss: 0.0742 - sparse_categorical_accuracy: 0.9779 - val_loss: 0.5237 - val_sparse_categorical_acc

In [6]:
start_time = time.time()
test_loss, test_acc = model.evaluate(dataset_test, verbose=0)
end_time = time.time()
training_duration = end_time - start_time
print(f"Evaluation time: {training_duration:.2f} seconds")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Evaluation time: 4.26 seconds
Test Loss: 0.3795
Test Accuracy: 86.82%


In [7]:
model.save('model_MobileNetV3.keras')

In [8]:
print("Baseline model in Mb:", os.path.getsize(r'model_MobileNetV3.keras') / float(2**20))

Baseline model in Mb: 4.177478790283203


In [9]:
loaded_model = tf.keras.models.load_model(r"model_MobileNetV3.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_MobileNetV3_quantized_grey.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpqxbztjcu\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpqxbztjcu\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpqxbztjcu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  3230010067088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010067472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010067280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010066128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010065552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010065936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010068816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010068624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010068240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230010069392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230

In [10]:
loaded_model = tf.keras.models.load_model("model_MobileNetV3.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_MobileNetV3_grey.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp9fejugxz\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp9fejugxz\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmp9fejugxz'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  3230652524944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284147536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284147152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3230652526096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284146576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284147344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284145232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284145040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284146768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232284144848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3232

In [ ]:
#MobileNetV3 trained on greyscale images
interpreter = tf.lite.Interpreter(model_path="model_MobileNetV3_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_MobileNetV3_grey.tflite') / float(2**20))

c:\Graduate_School\Fall_2025\ECE528\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite accuracy: 0.8681955061385221
TFlite model in Mb: 3.576263427734375


In [ ]:
#MobileNetV3 quantized trained on greyscale images
interpreter = tf.lite.Interpreter(model_path="model_MobileNetV3_quantized_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_MobileNetV3_quantized_grey.tflite') / float(2**20))

TFLite accuracy: 0.8554551772063933
TFlite model in Mb: 1.0618667602539062
